# Excerise - Logistic Regression and Classification Measures
In this exercise you will explore how hard it is to find the best balance between TPR and FPR. 

Next, you will implement a logistic regression classifier using gradient decent.

We will then turn to applying multi-class classifiers over the well known MNIST digits database, and analyse the results using a confusion matrix.

## Balancing TPR and FPR and interpretability of a classifier results given inbalance between classes
After your yearly checkup, the doctor has bad news and good news. The bad news is that you tested positive for a serious disease, and that the test is 99% accurate (i.e. the probability of testing positive given that you have the disease is 0.99, as is the probability of testing negative given that you don’t have the disease). The good news is that this is a rare disease, striking only 1 in 10,000 people. 

1. What is the accuracy of always predicting a patient is healthy? Would that be a good strategy?

2. What are the TPR and FPR of the chosen classifier in this case? Giving that there is a tradeoff between the two --- is it a good point on the ROC curve?

3. Why is it good news that the disease is rare? What are the chances that you actually have the disease?

4. Taking into account the previous answers, would you say the designer of the classifier made a good choice in this case?



#### Answer 1
Always predicting that a patient is healty mean that $TP=0$ and $FP=0$, given that we find that:

$Accuracy = \frac{TP+TN}{TP+TN+FP+FN} = \frac{TN}{TN+FN} = 0.99$

Predicting always healty can lead to good accuracy for the model, but in real life it will lead to  100 deads for every 1,000,000 people... SAD :-(

#### Answer 2

$ TPR=\frac{TP}{TP + FN}= 0.99$

$ FPR=\frac{FP}{TN + FP}= 0.01$

Yes thats a good point on the ROC, because the TPR is high meaning we classify sick correctly and vice versa for FPR (low)

#### Answer 3
The chances that one actually have the disease is:

$P(sick|positive) =(*) \frac{P(positive|sick)*P(sick)}{P(positive)} =(**) \frac{P(positive|sick)*P(sick)}{P(positive|sick)*P(sick) + P(positive|healty)*P(healty)} = 0.0098$

$\text{(*) Bayes' theorem},    \text{(**) Law of total probability}$

The good news is that even if one have tested positive the probability of him being sick is very low.

#### Answer 4
Yes

## Implementing Logistic Regression Model
Recall the loss function of the binary (Univariate) Logistic Regression model

$$ J(\beta)=-\frac{1}{m}\sum_{i=1}^{m}\left(y^{(i)}\log(h_\beta(x^{(i)}))+(1-y^{(i)})\log(1-h_\beta(x^{(i)}))\right) $$

and the gradient decent update rule is given by

$$ \beta_j^t := \beta_j^{t-1}-\mu\frac{\partial}{\partial \beta_j}J(\beta)
$$

1. Write the explicit (and rather simple) update rule for a gradient decent solver of the Logistic Regression model.

$\beta_j^t := \beta_j^{t-1}-\mu\frac{\partial}{\partial \beta_j}J(\beta) = \beta_j^{t-1}-\mu\sum_{i=1}^{m}(y^{(i)} - h_\beta(x^{(i)}))x_j^{(i)}$

2. Implement a LogisticRegression class with `fit`, `predict`, and `score` functions, and the constructor of the class should have the following parameters (feel free to add methods and parameters as you see fit):
  * `method`: either `GD` or `SGD`, indiacting whether to use a regular gradient decent, or an [iterative stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Iterative_method) respectively.
  * `learning_rate`: the learning rate to be used
  * `num_iterations`: number of iterations for the gradient decent algorithms (**note** that for stochastic gradient decent, each iteration is interpreted in that aspect as an epoch of going over all the samples, as it is computationaly similar to a single iteration of regular gradient decent)
  * `track_loss`: a boolean stating if to track the loss of the model for each iteration during training; if `True` the model will have a `loss` vector with the loss that was claculated for each of the iterations.

In [3]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import LabelBinarizer
import pandas as pd

In [4]:
class LogisticRegression:
    def __init__(self, method='GD', learning_rate=0.1, track_loss=True, iters=1000):
        self.method = method
        self.lr = learning_rate
        self.track = track_loss
        self.iters = iters
        self.w = None
        
    def fit(self, X, y):
        classes = np.array(list(set(y)))
        print('Found {} labels'.format(classes.size))
        lb = LabelBinarizer()
        lb.fit(y)
        y = lb.transform(y)
        # init weights
        w = np.zeros((X.shape[1], 1)) 
        for i in range(self.iters):
            w, cost = self.update_(X, y, w)
            if i % 100 == 0:
                print('iter: {}, cost: {}'.format(i, cost.sum()))

#         self.w = np.zeros((X.shape[1], classes.size))  
#         for c in np.arange(classes.size):
#             w = np.zeros((X.shape[1], classes.size))
#             for i in range(self.iters):
#                 w, cost = self.update_(X, y, w)
#                 if i % 100 == 0:
#                     print('iter: {}, cost: {}'.format(i, cost.sum()))
#             self.w = w
            
    def update_(self, X, y, w):
        m = y.shape[0]
        hx = self.sigmoid(X.dot(w))
        J = -1*(1/m)*(np.log(hx).T.dot(y) + np.log(1-hx).T.dot(1-y))
        #gradient = (1/m)*X.T.dot(hx - np.expand_dims(y, axis=1))
        gradient = (1/m)*X.T.dot(hx - y)
        w -= self.lr*gradient
        return w, J
    
    def predict(self, X):
        return self.sigmoid(X.dot(self.w))
    
    @staticmethod
    def sigmoid(z):
        return 1/(1+np.exp(-z))

3. Use the provided code to load the Iris dataset as a toy example, on which we will compare the results of the two logistic regression solvers. Run both solvers (GD and SGD) for 1000 iterations, using a learning rate of 0.1, and compare the learning curve by plotting the loss during training. Describe the results, and plot the decision boundary for each of the models. What would you see if you plot the loss after each parameters update (for each sample in every epoch)?
 
 
For simplicity, do not use regularization, and we will not try to find the best learning rate --- however, you are more than welcome to play with those on your own.

In [5]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X = np.c_[np.ones(X.shape[0]), X]    # bias padding

In [8]:
lgr = LogisticRegression()
lgr.fit(X[50:], y[50:])

Found 2 labels
iter: 0, cost: 0.6931471805599452
iter: 100, cost: 0.4372119701982426
iter: 200, cost: 0.3326355359640812
iter: 300, cost: 0.27778245442592214
iter: 400, cost: 0.24411233387156053
iter: 500, cost: 0.22125875407731235
iter: 600, cost: 0.2046550945551776
iter: 700, cost: 0.1919917748096203
iter: 800, cost: 0.1819773714547795
iter: 900, cost: 0.17383313409932125


** It seem that i have error when trying MultiClass **

## Classifying Digits
In this part we will test digits classification on the MNIST dataset, using Multivariate Logistic Regression (a discriminative model). 

The MNIST dataset contains 28x28 grayscale images of handwritten digits between 0 and 9 (10 classes). For mathmatical analysis clarity, and for matching expected API, each image faltten to create a 1D array with 784 elements.

### Loading the MNIST dataset
Load the MNIST data set. The digits dataset is one of datasets scikit-learn comes with that do not require the downloading of any file from some external website. Use 
>```mnist = sklearn.datasets.fetch_mldata('MNIST original')```

to fetch the original data. You may set the `data_home` to where you wish to download your data for caching. Each image is already transformed into a 1D integer array $x\in [0,255]^{784}$, and the corresponding label is an integer $y\in [0,9]$.

Plot a single sample of each digit as the original image, so you get a feeling how the data looks like.

Finally, divide your data into train and test sets, using 1/7 of the data for testing.

---
**Note:** Using `digits = sklearn.datasets.load_digits()` will only fetch a very small sample of the original set, with images resized to 8x8. This preprocessing of the data reduces dimensionality and gives invariance to small distortions - however, we will use the original data in this exercise. Feel free to test the proformance of the algorithms below on the preprocessed data as well.


In [384]:
from sklearn.datasets.base import get_data_home 
print (get_data_home())

C:\Users\217216X717997\scikit_learn_data


In [385]:
mnist = datasets.fetch_mldata('MNIST original')

c:\users\217216x717997\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
c:\users\217216x717997\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
mnist

### Multivariate Logistic Regression
We will now apply the Multivariate Logistic Regression model discriminative model for this problem. We will use a prepared implementation of the model, so you won't need to implement it.

In this part we will keep the pixel values in grayscale, and assume the probability of each image being classified as one of the 10 digit classes, matches some multivariate logistic function.

#### Question 1
Run a Multivariate Logitic Regression classifier on the training data and apply predictions on the test data. Use the [sklearn.linear_model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) implementation, and set the `solver` to be `'lbfgs'`, and  `multi_class` to `'multinomial'` (leave all other parameters in their default value).

1. Plot the confusion matrix of your classifier, as claculated on the test data (it is recommended to use [sklearn.metrics.confusion_matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)). Calculate the total accuracy (fraction of correctly classified images), and summarize the results in your own words.

2. Show some test images which were missclassified (classified incorrectly). Choose one such sample, and output the probability estimates for all classes (use the `predict_proba` function). Describe the results.

Note that here we cannot "generate" examples from the model.

#### Answer 1
Put you answer here...

In [ ]:
# code goes here

#### Question 2
Comparing two methods for solving multiclass classification problems: one-vs-rest (OvR), and multinomial.

1. Discuss the results, and explain the plots in [Plot multinomial and One-vs-Rest Logistic Regression](https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic_multinomial.html).

2. Create another Logistic Regression classifier, setting now `multi_class` to `'ovr'` (instead of `multinomial`). Leave all other parameters the same. Compare the accuracy of this model with the previous one model. 




#### Answer 2
Put you answer here...

In [ ]:
# code goes here